In [1]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/tree-survival-prediction/Tree_Data.csv', index_col=['No']).sort_index()
df.head()

,Plot,Subplot,Species,Light_ISF,Light_Cat,Core,Soil,Adult,Sterile,Conspecific,...,AMF,EMF,Phenolics,Lignin,NSC,Census,Time,Event,Harvest,Alive
No,,,,,,,,,,,,,,,,,,,,,
3,1,A,Prunus serotina,0.108,Med,2017,Prunus serotina,H,Non-Sterile,Conspecific,...,23.31,NaN,0.20,10.42,13.92,12,42.0,1.0,NaN,NaN
11,1,C,Quercus alba,0.106,Med,2017,Quercus rubra,970,Non-Sterile,Heterospecific,...,15.82,31.07,5.19,20.52,19.29,33,115.5,0.0,NaN,X
12,1,C,Quercus rubra,0.106,Med,2017,Prunus serotina,J,Non-Sterile,Heterospecific,...,24.45,28.19,3.36,24.74,15.01,18,63.0,1.0,NaN,NaN
14,1,C,Prunus serotina,0.106,Med,2017,Prunus serotina,G,Non-Sterile,Conspecific,...,35.60,NaN,0.37,10.22,13.87,16,56.0,1.0,NaN,NaN
18,1,C,Prunus serotina,0.106,Med,2016,Acer rubrum,1332,Non-Sterile,Heterospecific,...,35.29,NaN,0.30,10.80,13.79,7,24.5,1.0,NaN,NaN


In [2]:
df.shape

(2783, 23)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2783 entries, 3 to 7772
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Plot         2783 non-null   int64  
 1   Subplot      2783 non-null   object 
 2   Species      2783 non-null   object 
 3   Light_ISF    2783 non-null   float64
 4   Light_Cat    2783 non-null   object 
 5   Core         2783 non-null   int64  
 6   Soil         2783 non-null   object 
 7   Adult        2783 non-null   object 
 8   Sterile      2783 non-null   object 
 9   Conspecific  2783 non-null   object 
 10  Myco         2783 non-null   object 
 11  SoilMyco     2783 non-null   object 
 12  PlantDate    2783 non-null   object 
 13  AMF          2783 non-null   float64
 14  EMF          1283 non-null   float64
 15  Phenolics    2783 non-null   float64
 16  Lignin       2783 non-null   float64
 17  NSC          2783 non-null   float64
 18  Census       2783 non-null   int64  
 19  Time       

In [4]:
df.nunique()

Plot             18
Subplot           5
Species           4
Light_ISF        53
Light_Cat         3
Core              2
Soil              7
Adult            36
Sterile           2
Conspecific       3
Myco              2
SoilMyco          3
PlantDate        19
AMF             924
EMF             682
Phenolics       494
Lignin         1095
NSC             998
Census           22
Time             22
Event             2
Harvest           1
Alive             1
dtype: int64

In [5]:
df.columns

Index(['Plot', 'Subplot', 'Species', 'Light_ISF', 'Light_Cat', 'Core', 'Soil',
       'Adult', 'Sterile', 'Conspecific', 'Myco', 'SoilMyco', 'PlantDate',
       'AMF', 'EMF', 'Phenolics', 'Lignin', 'NSC', 'Census', 'Time', 'Event',
       'Harvest', 'Alive'],
      dtype='object')

In [6]:
from plotly.express import bar
for column in ['Plot', 'Subplot', 'Species', 'Light_ISF', 'Light_Cat', 'Core', 'Soil',
       'Adult', 'Sterile', 'Conspecific', 'Myco', 'SoilMyco', 'PlantDate', 
               'Census', 'Time', 'Event',]:
    bar(data_frame=df[column].value_counts().to_frame().reset_index(), x=column, y='count').show()

In [7]:
from plotly.express import scatter
scatter(data_frame=df, x='AMF', y='Phenolics')